# Session 1 Homework

from https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/01-intro/homework.md

solutions by Kristian Rother

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Question 1

What's the version of NumPy that you installed?

In [2]:
np.__version__

'1.19.4'

### Question 2

What's the version of Pandas?

In [3]:
pd.__version__

'1.1.4'

#### Get the data

In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2021-09-06 19:21:05--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1,4M) [text/plain]
Saving to: ‘data.csv.1’

data.csv.1          100%[===================>]   1,41M  1,72MB/s    in 0,8s    

2021-09-06 19:21:06 (1,72 MB/s) - ‘data.csv.1’ saved [1475504/1475504]



In [5]:
df = pd.read_csv('data.csv')
df.head(3)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350


In [6]:
df.shape

(11914, 16)

In [7]:
df.dtypes

Make                  object
Model                 object
Year                   int64
Engine Fuel Type      object
Engine HP            float64
Engine Cylinders     float64
Transmission Type     object
Driven_Wheels         object
Number of Doors      float64
Market Category       object
Vehicle Size          object
Vehicle Style         object
highway MPG            int64
city mpg               int64
Popularity             int64
MSRP                   int64
dtype: object

### Question 3

What's the average price of BMW cars in the dataset?

In [8]:
df.loc[df['Make']=='BMW', 'MSRP'].mean()

61546.76347305389

### Question 4

Select a subset of cars after year 2015. How many of them have missing values for Engine HP?

In [9]:
df2015 = df.loc[df['Year']>=2015]  # >= not > decided on available answers in the form
df2015.shape

(5995, 16)

In [10]:
df2015['Engine HP'].isna().sum()

51

### Question 5

* Calculate the average "Engine HP" in the dataset.
* Use the fillna method and to fill the missing values in "Engine HP" with the mean value from the previous step.
* Now, calcualte the average of "Engine HP" again.
* Has it changed?

Round both means before answering this questions. You can use the round function for that:

In [11]:
mean_before = df['Engine HP'].mean()
mean_before.round(2)

249.39

In [12]:
df['Engine HP-filled'] = df['Engine HP'].fillna(mean_before)

In [13]:
mean_after = df['Engine HP-filled'].mean()
mean_after.round(2)

249.39

In [14]:
assert mean_before == mean_after

### Question 6

* Select all the "Rolls-Royce" cars from the dataset.
* Select only columns "Engine HP", "Engine Cylinders", "highway MPG".

In [15]:
rr = df.loc[df['Make']=='Rolls-Royce']
rr = rr[['Engine HP', 'Engine Cylinders', 'highway MPG']]
rr.shape

(31, 3)

* Now drop all duplicated rows using drop_duplicates method (you should get a dataframe with 7 rows).

In [16]:
rolls = rr.drop_duplicates()
rolls.shape

(7, 3)

* Get the underlying NumPy array. Let's call it X.

In [17]:
X = rolls.values
X

array([[325.,   8.,  15.],
       [563.,  12.,  19.],
       [563.,  12.,  21.],
       [563.,  12.,  20.],
       [322.,  12.,  15.],
       [453.,  12.,  19.],
       [624.,  12.,  21.]])

* Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX.

In [18]:
XTX = np.dot(X.T, X)
XTX

array([[1.754801e+06, 3.965600e+04, 6.519600e+04],
       [3.965600e+04, 9.280000e+02, 1.500000e+03],
       [6.519600e+04, 1.500000e+03, 2.454000e+03]])

* Invert XTX.
* What's the sum of all the elements of the result?

In [19]:
INV = np.linalg.inv(XTX)

In [20]:
INV.sum().round(4)

0.0322

#### Check: Multiplication should result in identity matrix

In [21]:
np.matmul(INV, XTX).round(5)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [-0., -0.,  1.]])

### Questions 7

* Create an array y with values [1000, 1100, 900, 1200, 1000, 850, 1300].

In [22]:
y = np.array([1000, 1100, 900, 1200, 1000, 850, 1300], dtype=np.float32)

Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w.
What's the value of the first element of w?.

$w = \dfrac{X^Ty}{X^TX}$

In [23]:
INV.shape, X.shape

((3, 3), (7, 3))

In [24]:
result = np.dot(np.matmul(INV, X.T), y)
result

array([ 0.19989598, 31.02612262, 31.65378877])

*This resolves the Normal Equation*

#### Control

In [25]:
from sklearn.linear_model import LinearRegression

m = LinearRegression(fit_intercept=False).fit(X, y)
m.coef_, m.intercept_

(array([ 0.19989598, 31.02612262, 31.65378877]), 0.0)

In [26]:
assert np.all(m.coef_.round(3) == result.round(3))